<a href="https://colab.research.google.com/github/patricksabry/UTS_ML2019_ID98106545/blob/master/A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 Take Home Exam

##Question 3:

**Marketing or advertising companies would be very interested in being able to
predict whether a Twitter message will spread as a meme or not, and even better,
construct it so that it will spread. Why is this a hard problem to solve? Describe two
approaches using data analytics to predict whether a tweet will go viral or not. How
would you validate these approaches? Discuss the ethical and social consequence
of this study.**

# Introduction

# Current Solutions

# Ethical and Social Implications

# Conclusion

# References

# Draft area (Research notes)
